*Para executar cada célula, selecione-a e clique "ctrl + enter"*

In [25]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import time
import numpy as np
import pyautogui
import re
import pandas as pd
from IPython.display import display, Markdown

# Inputs

Coloque as datas inicial e final a serem buscadas no clipping (formato dd/mm/aaaa).

In [2]:
#Setar parâmetros
data_inicial = str(input('Data Inicial:'))
data_final = str(input('Data Final:'))

Data Inicial:17/01/2023
Data Final:07/02/2023


# Coleta de informações

Ao rodar a célula abaixo, abra a aba do Google Chrome que surgirá em seu computador e não mexa no seu mouse ou teclado até que ela se feche.

In [26]:
#Chegar na busca
nav = webdriver.Chrome()
nav.get(r'https://portalsinopress.com.br/sgc/identificador.php?_anterior=https%3A%2F%2Fportalsinopress.com.br%2Fsgc%2Findex.php%3F')
nav.find_element('xpath', '//*[@id="_email"]').send_keys('dleite@jeffreygroup.com', Keys.TAB)
nav.find_element('xpath', '//*[@id="_senha"]').send_keys('19411', Keys.ENTER)
nav.find_element('xpath', '//*[@id="moldura_meio_menu"]/a[1]/img').click()
time.sleep(3)

#Seleções - BHN
select_element = nav.find_element('name', '_id_pessoa')
select = Select(select_element)
select.select_by_visible_text('BLACKHAWK NETWORK')
time.sleep(3)
select_element = nav.find_element('name', '_id_classificacao')
select = Select(select_element)
select.select_by_visible_text('Blackhawk Network')
nav.find_element('xpath', '//*[@id="formulario_busca_avancada"]/span[2]/label[2]').click()

#Inserção de parâmetros padrão
nav.find_element('xpath', '//*[@id="_busca_data_inicial"]').send_keys(data_inicial)
nav.find_element('xpath', '//*[@id="_busca_data_final"]').send_keys(data_final)
time.sleep(3)
nav.find_element('xpath', '//*[@id="formulario_busca_avancada"]/a/img').click()
time.sleep(4)

#Coleta de dados
audiencia = []
valoracao = []
titulo = []
link = []
veiculo = []

s = 3
y_in = 300

#Criar loop que só para quando levanta exceção
while(s>0):
    try:
        nav.find_element('xpath', f'//*[@id="formulario_conteiner_clipping"]/div[{s}]/div[2]/h1/a').click()
        time.sleep(3)
        try:
            titulo1 = nav.find_element('xpath', '//*[@id="clipping_detalhe_01_grupo"]/div[1]/h1').get_attribute('innerText')
            titulo.append(titulo1)
        except NoSuchElementException:
            titulo1 = 'N/A'
            titulo.append(titulo1)
        try:
            valoracao1 = nav.find_element('xpath', '//*[@id="clipping_detalhe_01_grupo"]/div[2]/span[2]/h4').get_attribute('innerText')
            valoracao.append(valoracao1)
        except NoSuchElementException:
            valoracao1 = 'N/A'
            valoracao.append(valoracao1)
        try:
            audiencia1 = nav.find_element('xpath', '//*[@id="clipping_detalhe_01_grupo"]/div[2]/span[3]/h4').get_attribute('innerText')
            audiencia.append(audiencia1)
        except NoSuchElementException:
            audiencia1 = 0
            audiencia.append(audiencia1)
        link1 = nav.find_element('xpath', '//*[@id="clipping_detalhe_01_grupo"]/div[2]/span[1]/h4/a').get_attribute('href')
        link.append(link1)
        veiculo1 = nav.find_element('xpath', '//*[@id="clipping_detalhe_01_grupo"]/div[1]/h2').get_attribute('innerText')
        veiculo.append(veiculo1)
        s+=1
    
    except ElementClickInterceptedException:
        pyautogui.click(x=577, y=y_in)
        y_in += 50
    
    except NoSuchElementException:
        s = -1

nav.quit()

In [4]:
#Ajeitar os títulos
titulo_novo = []
fim = len(titulo) - 1
s = 0

while (s<=fim):
    nome = titulo[s]
    nome = nome[:-13]
    titulo_novo.append(nome)
    s+=1
    
print(titulo_novo)

['A nova jornada de compra de gift cards no turismo', 'Busca por gift cards cresce em 2022 e serviços de turismo mostram futuro promissor', 'CRESCE USO DE CARTÕES-PRESENTE DE TURISMO NO BRASIL!', 'Empresa de cartões-presente vê futuro no Turismo no Brasil', 'Gift cards e sua importância para o futuro do e-commerce', 'Gift cards para turismo ganham força no Brasil', 'Gift cards para turismo ganham força no Brasil', 'Gift cards para turismo ganham força no Brasil', 'Gift cards para turismo ganham força no Brasil', 'Gift cards para turismo ganham força no Brasil', 'Gift cards para turismo ganham força no Brasil', 'Gift Cards para turismo ganham força no Brasil - Jornal Folha do Litoral', 'Gift cards para turismo ganham força no Brasil - Tecflow', 'Gift cards para turismo ganham força no Brasil.', 'Gift cards para turismo ganham força no Brasil.', 'Gift cards vão do varejo para o turismo e ganham força no Brasil']


In [5]:
#Ajeitar Veículos
veiculo_novo = []
fim = len(veiculo) - 1
s = 0

while (s<=fim):
    nome = re.split('-', veiculo[s])
    nome = nome[0]
    veiculo_novo.append(nome)
    s+=1
    
print(veiculo_novo)

['Consumidor Moderno ', 'Tudo Celular ', 'Simone Garcia ', 'Panrotas On Line ', 'CardMonitor ', 'Acesso Wi', 'JP Turismo ', 'Revista Live Marketing ', 'CardMonitor ', 'Fotografando na Viagem ', 'Revista Empreende Online ', 'Folha do Litoral de Balneário Camboriú ', 'TecFlow ', 'Visite Brasília ', 'Cidade Brasil Turismo ', 'Eletros ']


In [28]:
#Ordenar por audiencia
df = pd.DataFrame(veiculo_novo, columns=['Veículo'])
df['Título'] = titulo_novo
df['Audiência'] = audiencia
df['Valoração'] = valoracao
df['Link'] = link

df['Audiência'] = df['Audiência'].astype(float)
df = df.sort_values(by='Audiência', ascending=False)

print(df)

                                    Veículo  \
1                             Tudo Celular    
3                         Panrotas On Line    
0                       Consumidor Moderno    
11  Folha do Litoral de Balneário Camboriú    
10                Revista Empreende Online    
13                         Visite Brasília    
4                              CardMonitor    
8                              CardMonitor    
15                                 Eletros    
2                            Simone Garcia    
7                   Revista Live Marketing    
5                                 Acesso Wi   
9                   Fotografando na Viagem    
12                                 TecFlow    
14                   Cidade Brasil Turismo    
6                               JP Turismo    

                                               Título  Audiência  \
1   Busca por gift cards cresce em 2022 e serviços...   265400.0   
3   Empresa de cartões-presente vê futuro no Turis...     9103.0

In [31]:
#Ajeitar Audiência
audiencia = df['Audiência'].tolist()
fim = len(audiencia) - 1
s = 0

while (s<=fim):
    audiencia[s] = f'{audiencia[s]:_.0f}'
    audiencia[s] = audiencia[s].replace('.', ',').replace('_', '.')
    s+=1

In [32]:
#voltar pra listas
valoracao = df['Valoração'].tolist()
link = df['Link'].tolist()
veiculo = df['Veículo'].tolist()
titulo = df['Título'].tolist()

# Resultado final

Copie o resultado da célula abaixo e cole no e-mail do relatório. Aí, basta acrescentar a análise e enviar!

In [33]:
#Imprimir resultado
fim = len(veiculo) - 1
s = 0
n = 1

while (s<=fim):
    display(Markdown(f'**{n}. {veiculo[s]}| {titulo[s]}**'))
    print('')
    display(Markdown(f'**Audiência:** {audiencia[s]}'))
    display(Markdown(f'**Valoração:** {valoracao[s]}'))
    print('')
    display(Markdown(f'Leia [aqui]({link[s]}).'))
    print('')
    s+=1
    n+=1

**1. Tudo Celular | Busca por gift cards cresce em 2022 e serviços de turismo mostram futuro promissor**

**Audiência:** 265.400

**Valoração:** R$ 257.288,58

Leia [aqui](https://www.tudocelular.com/mercado/noticias/n201114/gift-cards-para-turismo-ganham-forca-no-brasil.html).

**2. Panrotas On Line | Empresa de cartões-presente vê futuro no Turismo no Brasil**

...

Leia [aqui](https://www.cidadebrasilturismo.com.br/2023/01/gift-cards-para-turismo-ganham-forca-no.html).

**16. JP Turismo | Gift cards para turismo ganham força no Brasil**

**Audiência:** 0

**Valoração:** 2020

Leia [aqui](https://jpturismo.com.br/gift-cards-para-turismo-ganham-forca-no-brasil/).